# Deriving A Mathematical Formula

In this demo, we will use genetic programming to determine a good mathematical formula given a particular set of input and output data.

The library we are using is called `rgp` and is one of the genetic programming libraries for R. Unfortunately, it does appear to be in a defunct status, as there appear not to have been any updates to the code base since 2015.

In [ ]:
if(!require(rgp)) {
    install.packages("rgp", repos = "http://cran.us.r-project.org")
    library(rgp)
}

if(!require(igraph)) {
  install.packages("igraph", repos = "http://cran.us.r-project.org")
  library(igraph)
}

We will start out with a set of x and y values.  There's no easy formula linking x and y, so we could see a relatively complex result here.

In [ ]:
input_data <- data.frame(
    x = c(1, 2, 3, 4, 5, 6, 7),
    y = c(0, 26, 88, 186, 320, 490, 541)
)
plot(x,y)

We will use three basic functions.  There are some functions for this problem that will result in error, like division (e.g., divide by zero errors), so we will stick to the safest operators for this demo.

In [ ]:
function_set <- functionSet("+", "-", "*")

Instead of building up each of the individual variables, we will call the `symbolicRegression` function, which builds a regression problem.  This way, we don't need to define things like `inputVariableSet`, `constantFactorySet`, etc.  The fitness function is how closely the resulting formula matches to the data provided in the y value.

In [ ]:
result <- symbolicRegression(
    y ~ x,
    data = input_data,
    functionSet = function_set,
    stopCondition = makeStepsStopCondition(500)
)

Now we can plot our results against the actual (x,y) coordinates to see how close our best shot was.

In [ ]:
plot(input_data$y, col=1, type="l"); points(predict(result, newdata = input_data), col=2, type="l")

It's a decent guess, but not perfect by any stretch.  Let's take a look at the function the genetic program generated.

In [ ]:
best_result <- result$population[[which.min(result$fitnessValues)]]
best_result

The best result is a fairly complex polynomial.  It might be a little too complex to understand easily in this format, so we can use a tool called `igraph` to build a graph based on these results.

As a note, `rgp` has a function called `funcToIgraph()` which we would call, passing in `best_result`.  Unfortunately, if you try to call `funcToIgraph(best_result)`, you get an error.  This is because when `rgp` was written, `igraph` had some code to monkey with 0-based arrays instead of the 1-based arrays that R uses.  That code was subsequently removed, but `rgp` was not updated to handle this.  Therefore, we need to create our own function which replaces what `funcToIgraph(best_result)` did.

In [ ]:
exprToIgraph <- function(expr) {
  if (!require("igraph")) stop("exprToIgraph: Package 'igraph' not installed.")
  exprGraph <- exprToGraph(expr)
  exprIgraph <- graph(exprGraph$edges, n = length(exprGraph$vertices)) # igraph vertexes used to be counted from zero but no longer are!
  V(exprIgraph)$label <- exprGraph$vertices
  exprIgraph
}

With that change now in place, we can call `exprToIgraph` and build up our graph tree.

In [ ]:
final_graph <- exprToIgraph(body(best_result))
plot(final_graph, layout=layout_as_tree, vertex.color="orange", vertex.size=30, edge.arrow.size=0.1)

This graph probably makes the most sense if you can easily read Reverse Polish Notation.  Still, it's nice that we are able to visualize the outputs of a genetic program.  This is most useful when we build decision tree-like programs which use conditional logic to make decisions, instead of the mathematical formula derivation functions that we have created thus far.